# Geocoder

**import libraries**

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

**create dataframe**

In [2]:
from bs4 import BeautifulSoup

req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df = pd.read_html(str(table))

neighborhood=pd.DataFrame(df[0])
neighborhood.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.**

In [3]:
# Drop the rows with Borough of "Not assigned"
df = neighborhood.replace('Not assigned',np.nan, regex=True)
df.dropna(subset=["Borough"], axis=0, inplace=True)
df = df.reset_index(drop=True)
df.head(10)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
# Merge rows with the same Postcode
df_assigned = df.groupby(['Postal Code', 'Borough'], as_index=False).agg(lambda x: ', '.join(set(x)))

# Set 'Not assigned' Neigborhood to be equal to Borough
na_nb_idx =df_assigned['Neighborhood'] == 'Not assigned'
df_assigned.loc[na_nb_idx, 'Neighborhood'] = df_assigned[na_nb_idx].Borough

***check if 'M5A' example is done correctly***

In [7]:
df_assigned.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_assigned.loc[df_assigned['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"


***use the .shape method to print the number of rows***

In [8]:
df_assigned.shape

(103, 3)

***read geo data from a CSV need to rename a column so the merge will work***

In [9]:
import geopy

In [10]:
dfgeo=pd.read_csv('http://cocl.us/Geospatial_data')
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
dfgeo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


***merge 2 dataframes***

In [11]:
df2 = pd.merge(df_assigned, dfgeo, on='PostalCode', how='left')
df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
